In [4]:
import json
import logging
import os
import sys
from collections import defaultdict
from itertools import product
from pathlib import Path
from typing import Dict, List, Optional

import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow.parquet as pq
import seaborn as sns
import textalloc as ta
from IPython.display import display
from sklearn.dummy import DummyClassifier
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, f1_score, precision_score, r2_score, recall_score
from tqdm import tqdm

NOTEBOOK_PATH: Path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
PROJECT_DIR: Path = NOTEBOOK_PATH.parent.parent
sys.path.append(str(PROJECT_DIR))
import src.utils.custom_log as custom_log
import src.utils.json_util as json_util
from src._StandardNames import StandardNames
from src.evaluate._Data import Data
from src.load.LoadForClassification import RENAMER, LoadForClassification
from src.utils.PathChecker import PathChecker
from src.utils.Csv import Csv
from src.utils.set_rcparams import set_rcparams

os.chdir(PROJECT_DIR)
set_rcparams()

LOG: logging.Logger = logging.getLogger(__name__)
custom_log.init_logger(log_lvl=logging.INFO)
LOG.info("Log start, project directory is %s (exist: %s)", PROJECT_DIR, PROJECT_DIR.is_dir())

CHECK: PathChecker = PathChecker()
STR: StandardNames = StandardNames()

FIG_DIR: Path = CHECK.check_directory(PROJECT_DIR / "reports" / "figures", exit=False)
FIG_DIR /= NOTEBOOK_PATH.stem
FIG_DIR.mkdir(parents=True, exist_ok=True)
LOG.info("Figure directory is %s (exist: %s)", FIG_DIR, FIG_DIR.is_dir())

EXP_DIR:Path = CHECK.check_directory(PROJECT_DIR /"experiments")
DIR_SEARCH_STR:str = "2024-12-*-*-*-*_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200"


2024-12-20 09:47:20,401     INFO  MainProcess  MainThread Setting rcparams for matplotlib
2024-12-20 09:47:20,406     INFO  MainProcess  MainThread Using style src/visualization/dissertation.mplstyle
2024-12-20 09:47:20,408     INFO  MainProcess  MainThread Log start, project directory is /root/py_projects/aihiii (exist: True)
2024-12-20 09:47:20,410     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/reports/figures' exists
2024-12-20 09:47:20,412     INFO  MainProcess  MainThread Figure directory is /root/py_projects/aihiii/reports/figures/cnn_evaluation (exist: True)
2024-12-20 09:47:20,413     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/experiments' exists


In [3]:
sorted([d for d in EXP_DIR.glob(DIR_SEARCH_STR) if d.is_dir()])

[PosixPath('/root/py_projects/aihiii/experiments/2024-12-03-22-18-51_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200'),
 PosixPath('/root/py_projects/aihiii/experiments/2024-12-04-02-05-31_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200'),
 PosixPath('/root/py_projects/aihiii/experiments/2024-12-04-05-20-24_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200'),
 PosixPath('/root/py_projects/aihiii/experiments/2024-12-04-08-57-36_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200')]

In [11]:
def get_results() -> pd.DataFrame:
    results = []
    for i, exp_dir in enumerate(sorted([d for d in EXP_DIR.glob(DIR_SEARCH_STR) if d.is_dir()])):
        db = Csv(csv_path=exp_dir / STR.fname_results_csv).read().loc[-1]
        db.index = [i] * db.shape[0]
        db.index.name = "Repetition"
        db.set_index("Data", append=True, inplace=True)
        results.append(db)

    results = pd.concat(results)
    results = results.sort_index()
    return results


RESULTS: pd.DataFrame = get_results()
RESULTS

2024-12-20 09:52:59,903     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-03-22-18-51_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200/results.csv
2024-12-20 09:52:59,911     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-04-02-05-31_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200/results.csv
2024-12-20 09:52:59,920     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-04-05-20-24_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200/results.csv
2024-12-20 09:52:59,926     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-04-08-57-36_pure_cnn_95HIII_injury_criteria_from_doe_sobol_20240705_194200/results.csv


Chest_Deflection  Chest_VC  Chest_a3ms  \
Repetition Data                                            
0          Test           0.930964  0.935379    0.752336   
           Train          0.934874  0.939548    0.760042   
1          Test           0.930841  0.935137    0.752312   
           Train          0.934401  0.938941    0.758652   
2          Test           0.930595  0.935896    0.752560   
           Train          0.934059  0.939306    0.760212   
3          Test           0.930974  0.936166    0.752931   
           Train          0.930938  0.938168    0.757083   

                  Femur_Fz_Max_Compression  Head_HIC15  Head_a3ms  \
Repetition Data                                                     
0          Test                   0.926570    0.929684   0.933507   
           Train                  0.929361    0.938504   0.939475   
1          Test                   0.926677    0.928839   0.933345   
           Train                  0.929643    0.938829   0.939879   
2          Test                   0.926702    0.929065   0.932778   
           Train                  0.928955    0.939487   0.940182   
3          Test                   0.926941    0.928783   0.933818   
           Train                  0.928718    0.939352   0.940464   

                  Neck_Fx_Shear_Max  Neck_Fz_Max_Tension  Neck_My_Extension  
Repetition Data                                                              
0          Test            0.923776             0.889334           0.922085  
           Train           0.930212             0.893869           0.928494  
1          Test            0.924176             0.889213           0.922071  
           Train           0.929962             0.894286           0.927217  
2          Test            0.924169             0.888104           0.921924  
           Train           0.932259             0.894564           0.927095  
3          Test            0.924143             0.889082           0.921756  
           Train           0.929711             0.891907           0.926568

In [26]:
print("MEAN")
display(RESULTS.groupby("Data").median().round(4))

MEAN


,Chest_Deflection,Chest_VC,Chest_a3ms,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension
Data,,,,,,,,,
Test,0.9309,0.9356,0.7524,0.9267,0.9290,0.9334,0.9242,0.8891,0.9220
Train,0.9342,0.9391,0.7593,0.9292,0.9391,0.9400,0.9301,0.8941,0.9272


In [27]:
((RESULTS.groupby("Data").max() - RESULTS.groupby("Data").min())/2).round(4)

,Chest_Deflection,Chest_VC,Chest_a3ms,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension
Data,,,,,,,,,
Test,0.0002,0.0005,0.0003,0.0002,0.0005,0.0005,0.0002,0.0006,0.0002
Train,0.0020,0.0007,0.0016,0.0005,0.0005,0.0005,0.0013,0.0013,0.0010


In [55]:
((RESULTS.groupby("Data").max() - RESULTS.groupby("Data").min())/2).round(4).median(axis=1)

Data
Test     0.0003
Train    0.0010
dtype: float64

In [28]:
(RESULTS.groupby("Data").min()+(RESULTS.groupby("Data").max() - RESULTS.groupby("Data").min())/2).round(4)

,Chest_Deflection,Chest_VC,Chest_a3ms,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension
Data,,,,,,,,,
Test,0.9308,0.9357,0.7526,0.9268,0.9292,0.9333,0.924,0.8887,0.9219
Train,0.9329,0.9389,0.7586,0.9292,0.9390,0.9400,0.931,0.8932,0.9275


In [56]:
(RESULTS.groupby("Data").min()+(RESULTS.groupby("Data").max() - RESULTS.groupby("Data").min())/2).round(4).median(axis=1)

Data
Test     0.9268
Train    0.9310
dtype: float64

In [34]:
db_5 = Csv(csv_path=PROJECT_DIR / "experiments"/"2024-12-04-12-45-14_pure_cnn_05HIII_injury_criteria_from_doe_sobol_20240705_194200" / STR.fname_results_csv).read().loc[-1].set_index("Data")
db_5.round(2)

2024-12-20 10:53:19,710     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-04-12-45-14_pure_cnn_05HIII_injury_criteria_from_doe_sobol_20240705_194200/results.csv


,Chest_Deflection,Chest_VC,Chest_a3ms,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension
Data,,,,,,,,,
Train,0.25,0.35,0.91,0.66,0.93,0.85,0.80,0.72,0.58
Test,0.24,0.34,0.90,0.66,0.93,0.84,0.79,0.71,0.56


In [47]:
display((db_5.loc["Train"] - db_5.loc["Test"]).round(3))
((db_5.loc["Train"] - db_5.loc["Test"]).round(3)).max()

Chest_Deflection            0.005
Chest_VC                    0.004
Chest_a3ms                  0.002
Femur_Fz_Max_Compression    0.003
Head_HIC15                  0.003
Head_a3ms                   0.004
Neck_Fx_Shear_Max           0.010
Neck_Fz_Max_Tension         0.007
Neck_My_Extension           0.018
dtype: float64

0.018

In [51]:
display((RESULTS.loc[(slice(None), "Train"), :].droplevel("Data") - RESULTS.loc[(slice(None), "Test"), :].droplevel("Data")).round(3))
(RESULTS.loc[(slice(None), "Train"), :].droplevel("Data") - RESULTS.loc[(slice(None), "Test"), :].droplevel("Data")).round(3).max().max()

,Chest_Deflection,Chest_VC,Chest_a3ms,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension
Repetition,,,,,,,,,
0,0.004,0.004,0.008,0.003,0.009,0.006,0.006,0.005,0.006
1,0.004,0.004,0.006,0.003,0.010,0.007,0.006,0.005,0.005
2,0.003,0.003,0.008,0.002,0.010,0.007,0.008,0.006,0.005
3,-0.000,0.002,0.004,0.002,0.011,0.007,0.006,0.003,0.005


0.011

In [69]:
def get_classification_results() -> pd.DataFrame:
    search_str = "2024-12-0*-*-*-*_pure_cnn_*HIII_injury_criteria_classes_*_from_doe_sobol_20240705_194200"
    results = []
    for exp_dir in EXP_DIR.glob(search_str):
        db = Csv(csv_path=exp_dir / STR.fname_results_csv).read(index_cols=[0,1,2])
        db["Median"] = db.median(axis=1)
        db = db.loc[(slice(None), -1, slice(None)), :].copy().droplevel("Fold")
        db["Classes"] = int(exp_dir.stem.split("_")[-6])
        db[STR.perc] = int(exp_dir.stem.split("_")[3][:2])
        db.set_index(["Classes", STR.perc], append=True, inplace=True)
        results.append(db)
    
    return pd.concat(results).sort_index()

CLASS_RESULTS = get_classification_results()
CLASS_RESULTS

2024-12-21 21:49:57,696     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-05-10-23-46_pure_cnn_05HIII_injury_criteria_classes_3_from_doe_sobol_20240705_194200/results.csv
2024-12-21 21:49:57,716     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-05-14-03-03_pure_cnn_05HIII_injury_criteria_classes_5_from_doe_sobol_20240705_194200/results.csv
2024-12-21 21:49:57,725     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-05-17-56-14_pure_cnn_05HIII_injury_criteria_classes_7_from_doe_sobol_20240705_194200/results.csv
2024-12-21 21:49:57,735     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-05-07-11-35_pure_cnn_05HIII_injury_criteria_classes_2_from_doe_sobol_20240705_194200/results.csv
2024-12-21 21:49:57,743     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-06-10-27-24_pure_cnn_95HIII_injury_criteria_classes_3_from_doe_sobol_202

Chest_Deflection  Chest_VC  \
Class Data  Classes PERC                               
0     Test  2       5             0.964983  1.000000   
                    95            0.937905  0.967650   
            3       5             0.964983  0.999634   
                    95            0.939656  0.927311   
            5       5             0.962622  0.990306   
...                                    ...       ...   
5     Train 7       95            0.777060  0.746963   
6     Test  7       5             0.521900  0.990306   
                    95            0.770444  0.735603   
      Train 7       5             0.530528  0.990306   
                    95            0.777060  0.746963   

                          Femur_Fz_Max_Compression  Head_HIC15  Head_a3ms  \
Class Data  Classes PERC                                                    
0     Test  2       5                     0.999817    0.978699   0.979932   
                    95                    1.000000    0.962586   0.943600   
            3       5                     0.999634    0.949402   0.902459   
                    95                    0.967913    0.924027   0.925066   
            5       5                     0.999634    0.913731   0.861291   
...                                            ...         ...        ...   
5     Train 7       95                    0.931281    0.862721   0.861450   
6     Test  7       5                     0.999634    0.896986   0.854249   
                    95                    0.928539    0.854069   0.851920   
      Train 7       5                     0.999634    0.903952   0.862204   
                    95                    0.931281    0.862721   0.861450   

                          Neck_Fx_Shear_Max  Neck_Fz_Max_Tension  \
Class Data  Classes PERC                                           
0     Test  2       5              0.934249             0.973370   
                    95             0.995242             0.990454   
            3       5              0.818906             0.907871   
                    95             0.985848             0.984057   
            5       5              0.744802             0.759163   
...                                     ...                  ...   
5     Train 7       95             0.955197             0.963775   
6     Test  7       5              0.716736             0.755099   
                    95             0.952700             0.960185   
      Train 7       5              0.726060             0.771950   
                    95             0.955197             0.963775   

                          Neck_My_Extension    Median  
Class Data  Classes PERC                               
0     Test  2       5              0.998535  0.979315  
                    95             0.986207  0.976928  
            3       5              0.821870  0.928636  
                    95             0.960509  0.950083  
            5       5              0.757450  0.887511  
...                                     ...       ...  
5     Train 7       95             0.867990  0.865356  
6     Test  7       5              0.760765  0.807507  
                    95             0.856879  0.855474  
      Train 7       5              0.764929  0.817077  
                    95             0.867990  0.865356  

[68 rows x 9 columns]

In [70]:
CLASS_RESULTS.loc[(slice(None), "Test", 5, 95), :]

,,,,Chest_Deflection,Chest_VC,Femur_Fz_Max_Compression,Head_HIC15,Head_a3ms,Neck_Fx_Shear_Max,Neck_Fz_Max_Tension,Neck_My_Extension,Median
Class,Data,Classes,PERC,,,,,,,,,
0,Test,5,95,0.881064,0.839811,0.96868,0.876195,0.874692,0.964713,0.961716,0.900067,0.890566
1,Test,5,95,0.881064,0.839811,0.96868,0.876195,0.874692,0.964713,0.961716,0.900067,0.890566
2,Test,5,95,0.881064,0.839811,0.96868,0.876195,0.874692,0.964713,0.961716,0.900067,0.890566
3,Test,5,95,0.881064,0.839811,0.96868,0.876195,0.874692,0.964713,0.961716,0.900067,0.890566
4,Test,5,95,0.881064,0.839811,0.96868,0.876195,0.874692,0.964713,0.961716,0.900067,0.890566


In [129]:
CLASS_RESULTS.groupby(["Classes", STR.perc, "Data"]).min().round(2).reset_index().set_index(
    ["Classes", "PERC", "Data"]
).unstack("Classes").T.loc[
    (
        [
            "Head_HIC15",
            "Head_a3ms",
            "Neck_My_Extension",
            "Neck_Fz_Max_Tension",
            "Neck_Fx_Shear_Max",
            "Chest_Deflection",
            "Chest_VC",
            "Femur_Fz_Max_Compression",
            "Median",
        ],
        slice(None),
    ),
]

PERC                                5           95      
Data                              Test Train  Test Train
                         Classes                        
Head_HIC15               2        0.98  0.98  0.96  0.96
                         3        0.95  0.96  0.92  0.93
                         5        0.91  0.92  0.88  0.88
                         7        0.90  0.90  0.85  0.86
Head_a3ms                2        0.98  0.98  0.94  0.95
                         3        0.90  0.91  0.93  0.93
                         5        0.86  0.88  0.87  0.88
                         7        0.85  0.86  0.85  0.86
Neck_My_Extension        2        1.00  1.00  0.99  0.99
                         3        0.82  0.84  0.96  0.96
                         5        0.76  0.76  0.90  0.91
                         7        0.76  0.76  0.86  0.87
Neck_Fz_Max_Tension      2        0.97  0.97  0.99  0.99
                         3        0.91  0.91  0.98  0.98
                         5        0.76  0.77  0.96  0.96
                         7        0.76  0.77  0.96  0.96
Neck_Fx_Shear_Max        2        0.93  0.94  1.00  1.00
                         3        0.82  0.83  0.99  0.99
                         5        0.74  0.75  0.96  0.97
                         7        0.72  0.73  0.95  0.96
Chest_Deflection         2        0.96  0.96  0.94  0.94
                         3        0.96  0.96  0.94  0.94
                         5        0.96  0.96  0.88  0.89
                         7        0.52  0.53  0.77  0.78
Chest_VC                 2        1.00  1.00  0.97  0.97
                         3        1.00  1.00  0.93  0.93
                         5        0.99  0.99  0.84  0.85
                         7        0.99  0.99  0.74  0.75
Femur_Fz_Max_Compression 2        1.00  1.00  1.00  1.00
                         3        1.00  1.00  0.97  0.97
                         5        1.00  1.00  0.97  0.97
                         7        1.00  1.00  0.93  0.93
Median                   2        0.98  0.98  0.98  0.98
                         3        0.93  0.93  0.95  0.95
                         5        0.89  0.90  0.89  0.90
                         7        0.81  0.82  0.86  0.87

In [139]:
def get_channel_results() -> pd.DataFrame:
    search_str = "2024-12-*-*-*-*_pure_cnn_*HIII_channels_from_doe_sobol_20240705_194200"
    results = []
    for exp_dir in EXP_DIR.glob(search_str):
        db = Csv(csv_path=exp_dir / STR.fname_results_csv).read().loc[-1]
        db[STR.perc] = int(exp_dir.stem.split("_")[3][:2])
        db.set_index(["Data", STR.perc], inplace=True)
        results.append(db)
    
    return pd.concat(results).sort_index()

CH_RESULTS = get_channel_results()
CH_RESULTS


2024-12-22 12:37:34,319     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-10-18-25-52_pure_cnn_95HIII_channels_from_doe_sobol_20240705_194200/results.csv
2024-12-22 12:37:34,332     INFO  MainProcess  MainThread Read /root/py_projects/aihiii/experiments/2024-12-09-21-38-45_pure_cnn_05HIII_channels_from_doe_sobol_20240705_194200/results.csv


03CHST0000OCCUACRD  03CHST0000OCCUACXD  03CHST0000OCCUACYD  \
Data  PERC                                                               
Test  5               0.895586            0.891077            0.842530   
      95              0.884169            0.873285            0.798068   
Train 5               0.896143            0.891291            0.844654   
      95              0.885722            0.874137            0.800182   

            03CHST0000OCCUACZD  03CHST0000OCCUDSXD  03CHSTLOC0OCCUDSXD  \
Data  PERC                                                               
Test  5               0.823536            0.820810            0.966129   
      95              0.817930            0.916573            0.972008   
Train 5               0.820845            0.821416            0.966174   
      95              0.824385            0.916957            0.971370   

            03CHSTLOC0OCCUDSYD  03CHSTLOC0OCCUDSZD  03FEMRLE00OCCUFOZD  \
Data  PERC                                                               
Test  5               0.847659            0.960368            0.883105   
      95              0.885993            0.962912            0.932935   
Train 5               0.850530            0.959052            0.883456   
      95              0.890376            0.962009            0.933476   

            03FEMRRI00OCCUFOZD  ...  03HEADLOC0OCCUDSZD  03NECKUP00OCCUFOXD  \
Data  PERC                      ...                                           
Test  5               0.864947  ...            0.977583            0.846226   
      95              0.921512  ...            0.976975            0.825192   
Train 5               0.863885  ...            0.977356            0.848382   
      95              0.921824  ...            0.977838            0.820511   

            03NECKUP00OCCUFOZD  03NECKUP00OCCUMOYD  03PELV0000OCCUACXD  \
Data  PERC                                                               
Test  5               0.829021            0.772509            0.924601   
      95              0.844901            0.779718            0.942870   
Train 5               0.832369            0.775125            0.924719   
      95              0.845634            0.779542            0.942830   

            03PELV0000OCCUACYD  03PELV0000OCCUACZD  03PELVLOC0OCCUDSXD  \
Data  PERC                                                               
Test  5               0.884152            0.928352            0.977772   
      95              0.855743            0.930605            0.976535   
Train 5               0.886525            0.926141            0.977642   
      95              0.856426            0.930895            0.976442   

            03PELVLOC0OCCUDSYD  03PELVLOC0OCCUDSZD  
Data  PERC                                          
Test  5               0.928387            0.961844  
      95              0.919656            0.972269  
Train 5               0.929401            0.959983  
      95              0.921595            0.971331  

[4 rows x 26 columns]

In [141]:
CH_RESULTS.T.round(2).loc[
    [
        "03HEAD0000OCCUACRD",
        "03CHST0000OCCUACRD",
        "03CHST0000OCCUDSXD",
        "03NECKUP00OCCUMOYD",
        "03NECKUP00OCCUFOZD",
        "03NECKUP00OCCUFOXD",
        "03FEMRLE00OCCUFOZD",
        "03FEMRLE00OCCUFOZD",
    ]
]

Data                Test       Train      
PERC                  5     95    5     95
03HEAD0000OCCUACRD  0.88  0.87  0.88  0.87
03CHST0000OCCUACRD  0.90  0.88  0.90  0.89
03CHST0000OCCUDSXD  0.82  0.92  0.82  0.92
03NECKUP00OCCUMOYD  0.77  0.78  0.78  0.78
03NECKUP00OCCUFOZD  0.83  0.84  0.83  0.85
03NECKUP00OCCUFOXD  0.85  0.83  0.85  0.82
03FEMRLE00OCCUFOZD  0.88  0.93  0.88  0.93
03FEMRLE00OCCUFOZD  0.88  0.93  0.88  0.93